In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, losses

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()


In [3]:
# data normalized
x_train = x_train/255.
x_test  = x_test/255.

# reshape for model input
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [4]:
y_train = utils.to_categorical(y_train)
y_test  = utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [5]:
input_shape = (28,28,1) # img_rows, img_colums, color_channels
num_classes = 10

In [6]:
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(inputs)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# 2nd Conv layer        
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# Fully Connected layer        
x = layers.Flatten()(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0     

In [7]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train Model
history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 21s 41ms/step - loss: 0.3043 - accuracy: 0.9091 - val_loss: 0.0906 - val_accuracy: 0.9726
Epoch 2/20
469/469 [==============================] - 20s 43ms/step - loss: 0.0896 - accuracy: 0.9735 - val_loss: 0.0619 - val_accuracy: 0.9795
Epoch 3/20
469/469 [==============================] - 24s 50ms/step - loss: 0.0678 - accuracy: 0.9788 - val_loss: 0.0588 - val_accuracy: 0.9812
Epoch 4/20
469/469 [==============================] - 23s 48ms/step - loss: 0.0562 - accuracy: 0.9827 - val_loss: 0.0532 - val_accuracy: 0.9804
Epoch 5/20
469/469 [==============================] - 22s 46ms/step - loss: 0.0493 - accuracy: 0.9846 - val_loss: 0.0433 - val_accuracy: 0.9850
Epoch 6/20
469/469 [==============================] - 21s 45ms/step - loss: 0.0442 - accuracy: 0.9863 - val_loss: 0.0512 - val_accuracy: 0.9828
Epoch 7/20
469/469 [==============================] - 21s 45ms/step - loss: 0.0397 - accuracy: 0.9878 - val_loss: 0.0498 - val_accuracy:

In [9]:
# import pickle
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [10]:
model.save

<bound method Model.save of <keras.engine.functional.Functional object at 0x000002698036C9A0>>

In [11]:
score = model.evaluate(x_test, y_test)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

313/313 [==============================] - 2s 7ms/step - loss: 0.0434 - accuracy: 0.9872
Test loss:  0.04336513951420784
Test accuracy:  0.9872000217437744


In [12]:
y_pred = model.predict(x_test[0].reshape(-1,28,28,1)).argmax(axis=1)
print(y_pred)

[7]


In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("mnist_cnn_quantized.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmpcjcz0mgw\assets


216076

In [14]:
from tinymlgen import port
c_code = port(model, variable_name='mnist_cnn', pretty_print=True, optimize=False) 

with open('mnist_cnn.h', 'w') as f:
    print(c_code, file=f)

INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmpk0whtdi7\assets


INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmpk0whtdi7\assets


In [15]:
# load raw x_test
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    
with open('x_test.h', 'w') as f:
    print("unsigned char x_test_dat[784] = {", file=f)    
    for i in range(28):
        s = "       "
        for j in range(28):
            s+=str(x_test[0][i][j])+', ' # select x_test[0]
        print(s, file=f)
    print("};", file=f)
f.close()

In [16]:
!type x_test.h

unsigned char x_test_dat[784] = {
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 84, 185, 159, 151, 60, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 67, 114, 72, 114, 163, 227, 254, 225, 254, 254, 254, 250, 229, 254, 254, 

In [17]:
import os

import tensorflow as tf
from tensorflow import keras


In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0     

In [19]:
model.save_weights('./weights')

In [20]:
model.save("my_new_model")

INFO:tensorflow:Assets written to: my_new_model\assets


INFO:tensorflow:Assets written to: my_new_model\assets


In [21]:

tf.keras.layers.Layer.get_weights(model) 



[array([[[[-1.04110464e-01, -1.65203164e-06,  2.37884089e-01,
           -3.10237825e-01, -2.96923548e-01,  4.58352953e-01,
            1.92964405e-01,  2.32697412e-01,  3.48750055e-01,
            1.56002924e-01,  1.89486757e-01,  1.64793804e-01,
           -1.05638415e-01, -1.37733057e-01, -1.22031868e-01,
            4.51979227e-02]],
 
         [[-5.56163549e-01,  2.06370667e-01, -8.19364190e-03,
           -2.15480730e-01, -5.02859175e-01,  2.03963608e-01,
            4.43553552e-02,  2.26573437e-01, -2.12303370e-01,
           -8.50096419e-02, -9.81942751e-03,  9.79391262e-02,
            7.16361105e-02,  1.91655606e-01, -1.52698934e-01,
            6.53725415e-02]],
 
         [[-3.76587719e-01, -8.61649681e-03, -1.14633605e-01,
            2.09931418e-01,  5.46984002e-02, -2.64825940e-01,
            2.60255128e-01,  2.06935719e-01, -1.48496225e-01,
            6.19881339e-02,  3.32372159e-01,  2.41584629e-01,
            3.16724390e-01,  9.23223868e-02,  3.91283125e-01,
      

In [22]:
tflite_interpreter = tf.lite.Interpreter(model_path='mnist_cnn_quantized.tflite')
tflite_interpreter.allocate_tensors()

'''
Check input/output details
'''
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_input_1:0
shape: [ 1 28 28  1]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.float32'>
